In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
import json
from pandas.io.json import json_normalize
from mlxtend.frequent_patterns import association_rules

In [2]:
data = json.load(open('2017-12-13.json'))

In [3]:
rawDf = json_normalize(data['games'])
rawDf.head()

,added,card_history,coin,duration,hero,hero_deck,id,legend,mode,note,opponent,opponent_deck,rank,region,result,user_hash
0,2017-12-13T23:39:04Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",False,NaN,Hunter,None,99021668,None,ranked,None,Warlock,None,19.0,Europe,win,CA841F543E06FAC435C22C144C43B20A
1,2017-12-13T23:33:32Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",True,NaN,Hunter,Midrange,99021345,None,ranked,None,Hunter,Midrange,19.0,Europe,win,CA841F543E06FAC435C22C144C43B20A
2,2017-12-13T23:23:02Z,"[{'player': 'me', 'turn': 1, 'card': {'id': 'L...",False,NaN,Hunter,Midrange,99020769,None,ranked,None,Druid,Jade,19.0,Europe,win,CA841F543E06FAC435C22C144C43B20A
3,2017-12-13T21:37:51Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",False,NaN,Shaman,Midrange,99014209,None,ranked,None,Warrior,None,19.0,Europe,loss,CA841F543E06FAC435C22C144C43B20A
4,2017-12-13T21:25:12Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",False,NaN,Shaman,Token,99013402,None,ranked,None,Druid,Jade,18.0,Europe,loss,CA841F543E06FAC435C22C144C43B20A


In [4]:
rawDf['card_history'].iloc[0][14]

{'card': {'id': 'NEW1_031', 'mana': 3, 'name': 'Animal Companion'},
 'player': 'me',
 'turn': 7}

In [9]:
opponentCards = []
meCards = []
for game in rawDf['card_history']:
    meCardsPlayed = {}
    opponentCardsPlayed = {}
    dictSelect = {'me':meCardsPlayed, 'opponent':opponentCardsPlayed}
    for action in game:
        card = action['card']['name']
        dictionary = dictSelect[action['player']]
        dictionary[str(card)] = dictionary.get(card, 0) + 1
    
    meCards.append(meCardsPlayed)
    opponentCards.append(opponentCardsPlayed)

In [10]:
meDf = pd.DataFrame(meCards)#.add_suffix('_me')
opponentDf = pd.DataFrame(opponentCards)#.add_suffix('_opponent')
df = meDf.append(opponentDf)
df = df.fillna(0).astype('int')
df.head()

,A Light in the Darkness,Aberrant Berserker,Abomination,Abusive Sergeant,Abyssal Enforcer,Acherus Veteran,Acidic Swamp Ooze,Acolyte of Agony,Acolyte of Pain,Adaptation,...,Wondrous Wand,Wrath,Wretched Tiller,Wyrmrest Agent,"Xaril, Poisoned Mind","Y'Shaarj, Rage Unbound",Young Priestess,Youthful Brewmaster,Ysera,Ysera Awakens
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
frequentSets = apriori(df, min_support=0.05, use_colnames=True)

In [12]:
rules = association_rules(frequentSets, metric='confidence', min_threshold=0.6)

KeyError: frozenset({'Lesser Heal', 'Acolyte of Pain'})

In [16]:
df.columns.dtype

dtype('O')